## CBoW మోడల్ శిక్షణ

ఈ నోట్‌బుక్ [AI for Beginners Curriculum](http://aka.ms/ai-beginners) యొక్క భాగం

ఈ ఉదాహరణలో, మనం CBoW భాషా మోడల్‌ను శిక్షణ ఇచ్చి మన స్వంత Word2Vec ఎంబెడ్డింగ్ స్థలాన్ని పొందడం గురించి చూడబోతున్నాము. టెక్స్ట్ మూలంగా AG News డేటాసెట్‌ను ఉపయోగిస్తాము.


In [ ]:
import torch
import torchtext
import os
import collections
import builtins
import random
import numpy as np

In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

ముందుగా మన డేటాసెట్‌ను లోడ్ చేసి టోకనైజర్ మరియు శబ్దకోశాన్ని నిర్వచిద్దాం. లెక్కింపులను కొంత పరిమితం చేయడానికి `vocab_size` ను 5000 గా సెట్ చేస్తాము.


In [ ]:
def load_dataset(ngrams = 1, min_freq = 1, vocab_size = 5000 , lines_cnt = 500):
    tokenizer = torchtext.data.utils.get_tokenizer('basic_english')
    print("Loading dataset...")
    test_dataset, train_dataset  = torchtext.datasets.AG_NEWS(root='./data')
    train_dataset = list(train_dataset)
    test_dataset = list(test_dataset)
    classes = ['World', 'Sports', 'Business', 'Sci/Tech']
    print('Building vocab...')
    counter = collections.Counter()
    for i, (_, line) in enumerate(train_dataset):
        counter.update(torchtext.data.utils.ngrams_iterator(tokenizer(line),ngrams=ngrams))
        if i == lines_cnt:
            break
    vocab = torchtext.vocab.Vocab(collections.Counter(dict(counter.most_common(vocab_size))), min_freq=min_freq)
    return train_dataset, test_dataset, classes, vocab, tokenizer

In [ ]:
train_dataset, test_dataset, _, vocab, tokenizer = load_dataset()

Loading dataset...
Building vocab...


In [ ]:
def encode(x, vocabulary, tokenizer = tokenizer):
    return [vocabulary[s] for s in tokenizer(x)]

## CBoW మోడల్

CBoW ఒక పదాన్ని దాని చుట్టుపక్కల ఉన్న $2N$ పదాల ఆధారంగా అంచనా వేయడం నేర్చుకుంటుంది. ఉదాహరణకు, $N=1$ అయితే, *I like to train networks* వాక్యం నుండి క్రింది జంటలు వస్తాయి: (like,I), (I, like), (to, like), (like,to), (train,to), (to, train), (networks, train), (train,networks). ఇక్కడ, మొదటి పదం ఇన్‌పుట్‌గా ఉపయోగించే పొరుగువారి పదం, రెండవది మనం అంచనా వేయాల్సిన పదం.

తర్వాతి పదాన్ని అంచనా వేయడానికి నెట్‌వర్క్ నిర్మించాలంటే, పొరుగువారి పదాన్ని ఇన్‌పుట్‌గా ఇస్తూ, పద సంఖ్యను అవుట్‌పుట్‌గా పొందాలి. CBoW నెట్‌వర్క్ నిర్మాణం ఇలా ఉంటుంది:

* ఇన్‌పుట్ పదం embedding లేయర్ ద్వారా పంపబడుతుంది. ఈ embedding లేయర్ మన Word2Vec embedding అవుతుంది, కాబట్టి దీన్ని ప్రత్యేకంగా `embedder` అనే వేరియబుల్‌గా నిర్వచిస్తాము. ఈ ఉదాహరణలో embedding పరిమాణం = 30 గా ఉపయోగిస్తాము, మీరు ఎక్కువ డైమెన్షన్లతో ప్రయోగించవచ్చు (నిజమైన word2vec 300 పరిమాణం కలిగి ఉంటుంది)
* embedding వెక్టర్ తరువాత ఒక లీనియర్ లేయర్‌కు పంపబడుతుంది, ఇది అవుట్‌పుట్ పదాన్ని అంచనా వేస్తుంది. అందుకే దీని వద్ద `vocab_size` న్యూరాన్లు ఉంటాయి.

అవుట్‌పుట్ కోసం, మనం `CrossEntropyLoss` ను లాస్ ఫంక్షన్‌గా ఉపయోగిస్తే, అంచనా ఫలితాలుగా పదాల సంఖ్యలను మాత్రమే ఇవ్వాలి, one-hot ఎంకోడింగ్ అవసరం లేదు.


In [ ]:
vocab_size = len(vocab)

embedder = torch.nn.Embedding(num_embeddings = vocab_size, embedding_dim = 30)
model = torch.nn.Sequential(
    embedder,
    torch.nn.Linear(in_features = 30, out_features = vocab_size),
)

print(model)

Sequential(
  (0): Embedding(5002, 30)
  (1): Linear(in_features=30, out_features=5002, bias=True)
)


## శిక్షణ డేటాను సిద్ధం చేయడం

ఇప్పుడు టెక్స్ట్ నుండి CBoW పద జంటలను లెక్కించే ప్రధాన ఫంక్షన్‌ను ప్రోగ్రామ్ చేద్దాం. ఈ ఫంక్షన్ విండో సైజ్‌ను నిర్దేశించడానికి అనుమతిస్తుంది, మరియు ఇన్‌పుట్ మరియు అవుట్‌పుట్ పదాల జంటలను తిరిగి ఇస్తుంది. ఈ ఫంక్షన్ పదాలపై మాత్రమే కాకుండా వెక్టర్లు/టెన్సర్లపై కూడా ఉపయోగించవచ్చు - ఇది టెక్స్ట్‌ను ఎన్‌కోడ్ చేసి, తరువాత `to_cbow` ఫంక్షన్‌కు పంపడానికి సహాయపడుతుంది.


In [ ]:
def to_cbow(sent,window_size=2):
    res = []
    for i,x in enumerate(sent):
        for j in range(max(0,i-window_size),min(i+window_size+1,len(sent))):
            if i!=j:
                res.append([sent[j],x])
    return res

print(to_cbow(['I','like','to','train','networks']))
print(to_cbow(encode('I like to train networks', vocab)))

[['like', 'I'], ['to', 'I'], ['I', 'like'], ['to', 'like'], ['train', 'like'], ['I', 'to'], ['like', 'to'], ['train', 'to'], ['networks', 'to'], ['like', 'train'], ['to', 'train'], ['networks', 'train'], ['to', 'networks'], ['train', 'networks']]
[[232, 172], [5, 172], [172, 232], [5, 232], [0, 232], [172, 5], [232, 5], [0, 5], [1202, 5], [232, 0], [5, 0], [1202, 0], [5, 1202], [0, 1202]]


ప్రశిక్షణ డేటాసెట్‌ను సిద్ధం చేద్దాం. మనం అన్ని వార్తలను పరిశీలించి, పదజంటల జాబితా పొందడానికి `to_cbow` ను పిలవబోతున్నాము, ఆ జంటలను `X` మరియు `Y` లో చేర్చుతాము. సమయాన్ని దృష్టిలో ఉంచుకుని, మొదటి 10,000 వార్తలే పరిగణలోకి తీసుకుంటాము - మీకు ఎక్కువ సమయం ఉంటే, ఈ పరిమితిని సులభంగా తొలగించి, మెరుగైన ఎంబెడ్డింగ్స్ పొందవచ్చు :)


In [ ]:
X = []
Y = []
for i, x in zip(range(10000), train_dataset):
    for w1, w2 in to_cbow(encode(x[1], vocab), window_size = 5):
        X.append(w1)
        Y.append(w2)

X = torch.tensor(X)
Y = torch.tensor(Y)

మేము ఆ డేటాను ఒక డేటాసెట్‌గా మార్చి, డేటాలోడర్‌ను సృష్టిస్తాము:


In [ ]:
class SimpleIterableDataset(torch.utils.data.IterableDataset):
    def __init__(self, X, Y):
        super(SimpleIterableDataset).__init__()
        self.data = []
        for i in range(len(X)):
            self.data.append( (Y[i], X[i]) )
        random.shuffle(self.data)

    def __iter__(self):
        return iter(self.data)

మేము ఆ డేటాను ఒక dataset గా మార్చి, dataloader ను సృష్టిస్తాము:


In [ ]:
ds = SimpleIterableDataset(X, Y)
dl = torch.utils.data.DataLoader(ds, batch_size = 256)

ఇప్పుడు నిజమైన శిక్షణను చేద్దాం. మనం `SGD` ఆప్టిమైజర్‌ను చాలా ఎక్కువ లెర్నింగ్ రేట్‌తో ఉపయోగించబోతున్నాము. మీరు ఇతర ఆప్టిమైజర్లతో కూడా ఆడుకోవచ్చు, ఉదాహరణకు `Adam`. మొదట 10 epochs కోసం శిక్షణ ఇస్తాము - మరియు మీరు ఇంకా తక్కువ నష్టం కావాలనుకుంటే ఈ సెల్‌ను మళ్లీ నడపవచ్చు.


In [ ]:
def train_epoch(net, dataloader, lr = 0.01, optimizer = None, loss_fn = torch.nn.CrossEntropyLoss(), epochs = None, report_freq = 1):
    optimizer = optimizer or torch.optim.Adam(net.parameters(), lr = lr)
    loss_fn = loss_fn.to(device)
    net.train()

    for i in range(epochs):
        total_loss, j = 0, 0, 
        for labels, features in dataloader:
            optimizer.zero_grad()
            features, labels = features.to(device), labels.to(device)
            out = net(features)
            loss = loss_fn(out, labels)
            loss.backward()
            optimizer.step()
            total_loss += loss
            j += 1
        if i % report_freq == 0:
            print(f"Epoch: {i+1}: loss={total_loss.item()/j}")

    return total_loss.item()/j

In [ ]:
train_epoch(net = model, dataloader = dl, optimizer = torch.optim.SGD(model.parameters(), lr = 0.1), loss_fn = torch.nn.CrossEntropyLoss(), epochs = 10)

Epoch: 1: loss=5.664632366860172
Epoch: 2: loss=5.632101973960962
Epoch: 3: loss=5.610399051405015
Epoch: 4: loss=5.594621561080262
Epoch: 5: loss=5.582538017415446
Epoch: 6: loss=5.572900234519603
Epoch: 7: loss=5.564951676341915
Epoch: 8: loss=5.558288112064614
Epoch: 9: loss=5.552576955031129
Epoch: 10: loss=5.547634165194347


5.547634165194347

## Word2Vec ను ప్రయత్నించడం

Word2Vec ఉపయోగించడానికి, మన పదసంపదలోని అన్ని పదాలకు సంబంధించిన వెక్టార్లను తీసుకుందాం:


In [ ]:
vectors = torch.stack([embedder(torch.tensor(vocab[s])) for s in vocab.itos], 0)

ఉదాహరణకు, **Paris** అనే పదం ఎలా వెక్టర్‌గా ఎన్‌కోడ్ చేయబడిందో చూద్దాం:


In [ ]:
paris_vec = embedder(torch.tensor(vocab['paris']))
print(paris_vec)

tensor([-0.0915,  2.1224, -0.0281, -0.6819,  1.1219,  0.6458, -1.3704, -1.3314,
        -1.1437,  0.4496,  0.2301, -0.3515, -0.8485,  1.0481,  0.4386, -0.8949,
         0.5644,  1.0939, -2.5096,  3.2949, -0.2601, -0.8640,  0.1421, -0.0804,
        -0.5083, -1.0560,  0.9753, -0.5949, -1.6046,  0.5774],
       grad_fn=<EmbeddingBackward>)


సమానార్థక పదాలను వెతకడానికి Word2Vec ఉపయోగించడం ఆసక్తికరం. క్రింది ఫంక్షన్ ఇచ్చిన ఇన్‌పుట్‌కు అత్యంత సమీపమైన `n` పదాలను తిరిగి ఇస్తుంది. వాటిని కనుగొనడానికి, మనం $|w_i - v|$ యొక్క నార్మ్‌ను లెక్కిస్తాము, ఇక్కడ $v$ మన ఇన్‌పుట్ పదానికి సంబంధించిన వెక్టర్, మరియు $w_i$ శబ్దకోశంలో i-వ పదం యొక్క ఎంకోడింగ్. ఆపై మేము ఆ అర్రేను సర్ది, `argsort` ఉపయోగించి సంబంధిత సూచికలను తిరిగి ఇస్తాము, మరియు మొదటి `n` అంశాలను తీసుకుంటాము, ఇవి శబ్దకోశంలో సమీప పదాల స్థానాలను సూచిస్తాయి.


In [ ]:
def close_words(x, n = 5):
  vec = embedder(torch.tensor(vocab[x]))
  top5 = np.linalg.norm(vectors.detach().numpy() - vec.detach().numpy(), axis = 1).argsort()[:n]
  return [ vocab.itos[x] for x in top5 ]

close_words('microsoft')

['microsoft', 'quoted', 'lp', 'rate', 'top']

In [ ]:
close_words('basketball')

['basketball', 'lot', 'sinai', 'states', 'healthdaynews']

In [ ]:
close_words('funds')

['funds', 'travel', 'sydney', 'japan', 'business']

## ముఖ్యాంశం

CBoW వంటి తెలివైన సాంకేతికతలను ఉపయోగించి, మనం Word2Vec మోడల్‌ను శిక్షణ ఇవ్వవచ్చు. మీరు సెంట్రల్ పదం ఇచ్చినప్పుడు పక్కన ఉన్న పదాన్ని అంచనా వేయడానికి శిక్షణ పొందిన skip-gram మోడల్‌ను కూడా శిక్షణ ఇవ్వడానికి ప్రయత్నించవచ్చు, మరియు అది ఎంత బాగా పనిచేస్తుందో చూడవచ్చు.


---

<!-- CO-OP TRANSLATOR DISCLAIMER START -->
**అస్పష్టత**:  
ఈ డాక్యుమెంట్‌ను AI అనువాద సేవ [Co-op Translator](https://github.com/Azure/co-op-translator) ఉపయోగించి అనువదించబడింది. మేము ఖచ్చితత్వానికి ప్రయత్నించినప్పటికీ, ఆటోమేటెడ్ అనువాదాల్లో పొరపాట్లు లేదా తప్పిదాలు ఉండవచ్చు. మూల డాక్యుమెంట్ దాని స్వదేశీ భాషలో అధికారిక మూలంగా పరిగణించాలి. ముఖ్యమైన సమాచారానికి, ప్రొఫెషనల్ మానవ అనువాదం సిఫార్సు చేయబడుతుంది. ఈ అనువాదం వాడకంలో ఏర్పడిన ఏవైనా అపార్థాలు లేదా తప్పుదారితీసే అర్థాలు కోసం మేము బాధ్యత వహించము.
<!-- CO-OP TRANSLATOR DISCLAIMER END -->
